# Log messages

[![Click and Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/metadriverse/metadrive/blob/main/documentation/source/log_msg.ipynb)


Logging message is important to oversee the program status. MetaDrive adopts Python's `logging` module to log messages. 

## Global logger 
We provide a wrapper in `metadrive/engine/logger` to customize some features. We recommend using it to log messages to make the logging consistent. It is a global variable, so you can get the logger at anywhere of your code. Some classes like `BaseEnv` may have a variable called `logger` as well, which is actually the same as the global logger.

In [ ]:
from metadrive.envs.base_env import BaseEnv
from metadrive.engine.logger import get_logger # get_logger

logger = get_logger()

# create environment
logger.info("Create environment\n")
env = BaseEnv(dict(use_render=False))
assert logger is env.logger # the environment.logger is the same as global logger

# reset environment
logger.info("Reset environment")
env.reset()
try:
    for i in range(5):
        logger.info("Simulate step {}".format(i))
        env.step(env.action_space.sample())
finally:
    logger.info("Close environment")
    env.close()


The logger can be used at everywhere. For example, it can be used in your manager.

In [ ]:
from metadrive.envs.base_env import BaseEnv
from metadrive.engine.logger import get_logger # get_logger
from metadrive.manager.base_manager import BaseManager

logger = get_logger()

class MyMgr(BaseManager):
    
    def __init__(self):
        super(MyMgr, self).__init__()
        logger.info("Init MyMgr...")
    
    def after_step(self, *args, **kwargs):
        logger.info("Step {}...".format(self.episode_step))
        return dict()


# create environment
env = BaseEnv(dict(use_render=False))

# reset environment
env.reset()

# add manager
env.engine.register_manager("my_mgr", MyMgr())
try:
    for i in range(5):
        env.step(env.action_space.sample())
finally:
    env.close()

## Log level
Similar to `logging` module, one can change the log level to decide which log messages to be printed. Generally, MetaDrive uses `logging.INFO`, `logging.DEBUG`, and `logging.WARNING` to classify log messages. By default, the log level is `logging.INFO` and thus debug messages will be hidden. The log level can be set in environment config. The following example shows how to print more debug message by setting `log_level=logging.DEBUG`. In addition, if `debug=True` in the environment config, the log level will be set to `logging.DEBUG` as well. 

In [ ]:
from metadrive.envs.base_env import BaseEnv
import logging
from metadrive.engine.logger import get_logger # get_logger

logger = get_logger()

# create environment
env = BaseEnv(dict(use_render=False, log_level=logging.DEBUG))
assert logger is env.logger # the environment.logger is the same as global logger

# reset environment
logger.debug("Reset environment")
env.reset()
try:
    for i in range(5):
        logger.debug("Simulate step {}".format(i))
        env.step(env.action_space.sample())
finally:
    logger.debug("Close environment")
    env.close()

If you would like to turn off all log messages, changing the log_level to `logging.WARNING` (or `logging.CRITICAL`) can suppress messages logged by `logging.info()`, `logging.debug()` (or even `logging.warn`).

In [ ]:
from metadrive.engine.logger import set_log_level # get_logger

def run(): # define the simulation loop
    # reset environment
    logger.debug("Reset environment")
    env.reset()
    try:
        for i in range(5):
            logger.debug("Simulate step {}".format(i))
            env.step(env.action_space.sample())
    finally:
        logger.debug("Close environment")
        env.close()

set_log_level(logging.WARNING) # Or set it in environment config
run()


print("Turn off all messages...")    

set_log_level(logging.CRITICAL) # Or set it in environment config
run()

## Log once
Sometimes, we may want the message to be logged only once in one episode. In this case, we can use the `log_once` key to notify the logger to show some messages only once until the next `reset` is called.

In [ ]:
from metadrive.envs.base_env import BaseEnv
import logging

# create environment
env = BaseEnv(dict(use_render=False, log_level=logging.CRITICAL))

# reset environment
env.reset()
try:
    for i in range(5):
        env.logger.critical("Step... (I will only show once)".format(i), extra={"log_once": True})
        env.step(env.action_space.sample())
finally:
    env.close()
